In [ ]:
#| default_exp chatbot

In [ ]:
#| export
import ipywidgets as widgets
import traitlets
from ipywidgets import Textarea, Text, Layout, HBox
from traitlets import HasTraits
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI

In [ ]:
# read openapi key and set model
with open('OPENAI_API_KEY', 'r') as file:
    openai_api_key = file.read().strip()

llm = ChatOpenAI(openai_api_key = openai_api_key, model_name="gpt-3.5-turbo")

In [ ]:
#| export
class ChatBotModel(HasTraits):

    def __init__(self, llm, bot_template="You are playing the role of a tutor/educator"):
        super().__init__()
        self.llm = llm
        self.human_template = "{input_text}"
        self.update_bot_template(bot_template)

    def update_bot_template(self, bot_template):
        self.bot_template = bot_template
        self.bot_message_prompt = SystemMessagePromptTemplate.from_template(self.bot_template)
        self.human_message_prompt = HumanMessagePromptTemplate.from_template(self.human_template)
        self.chat_prompt = ChatPromptTemplate.from_messages([self.bot_message_prompt, self.human_message_prompt])
        
    def prompt(self, input_text):
        #prompt_val = prompt_template.invoke({"adjective": "funny", "content": "chickens"})
        ret = self.llm.invoke(self.chat_prompt.format_prompt(input_text=input_text))
        return ret.content

In [ ]:
chatbot_model = ChatBotModel(llm)
chatbot_model.prompt('hi')

In [ ]:
#| export

class ChatBotView(widgets.VBox):
    
    def __init__(self):
        # If you forget to call the superconstructor on an extended widget
        # you will get an AttributeError: object has no attribute '_model_id'
        super().__init__()

        self.chat = Textarea(
            disabled = True,
            layout=Layout(width='90%', height='400px')
        )
        self.user_input_and_submit = HBox()
        self.user_input = widgets.Text(
            placeholder='Message AI chatbot...',
            #layout=Layout(width='100%')
        )
        self.submit_button = widgets.Button(
            value=False,
            disabled=False,
            button_style='success',
            icon='arrow-circle-right' 
        )
        self.user_input_and_submit.children = (self.user_input, self.submit_button)

        self.children = (self.chat, self.user_input_and_submit) 

In [ ]:
chatbot_view = ChatBotView()
chatbot_view

In [ ]:
#| export
class ChatBot(ChatBotView):

    def __init__(self, model):
        # If you forget to call the superconstructor on an extended widget
        # you will get an AttributeError: object has no attribute '_model_id'
        super().__init__()
        self.submit_button.on_click(self.on_click)
        self.model =model

    def on_click(self, change):
        self.chat.value = self.chat.value + "USER: " + self.user_input.value + '\n\n'
        self.user_input.value = ''
        ret = self.model.prompt(self.user_input.value)
        self.chat.value = self.chat.value +  "CHATBOT: "  + ret + '\n\n'

In [ ]:
chatbot = ChatBot(chatbot_model)
chatbot

In [ ]:
#| export
class StudentChatBot(widgets.VBox):

    #user = traitlets.CUnicode()
    #response = traitlets.CUnicode()
    #step_by_step = traitlets.Bool()
    #metaphor = traitlets.Bool()
    #hints = traitlets.Bool()
    #guided_questions = traitlets.Bool() 
    
    def __init__(self, chatbot_model):
        # If you forget to call the superconstructor on an extended widget
        # you will get an AttributeError: object has no attribute '_model_id'
        super().__init__()

        self.chat_bot = ChatBot(chatbot_model)

        self.suggestion_buttons = HBox()
        self.step_by_step = widgets.Button(
            description='Step-by-Step',
            button_style='info',
            tooltip='Description',
        )
        self.metaphor = widgets.Button(
            description='Metaphor',
            button_style='info',
            tooltip='Description',
        )
        self.hints = widgets.Button(
            description='Hints',
            button_style='info',
            tooltip='Description',
        )
        self.guided_questions = widgets.Button(
            description='AI Guided Questions',
            button_style='info',
            tooltip='Description', 
        )
        self.suggestion_buttons.children = (self.step_by_step, self.metaphor, self.hints, self.guided_questions)
        self.children = (self.chat_bot, self.suggestion_buttons)

In [ ]:
student_chatbot = StudentChatBot(chatbot_model)
student_chatbot

In [ ]:
#| export
class EducatorChatBot(widgets.VBox):

    #user = traitlets.CUnicode()
    #response = traitlets.CUnicode()
    #step_by_step = traitlets.Bool()
    #metaphor = traitlets.Bool()
    #hints = traitlets.Bool()
    #guided_questions = traitlets.Bool()  
    
    def __init__(self, chatbot_model):
        # If you forget to call the superconstructor on an extended widget
        # you will get an AttributeError: object has no attribute '_model_id'
        super().__init__()

        self.chat_bot = ChatBot(chatbot_model)

        self.suggestion_buttons = HBox()
        self.exam_questions = widgets.Button(
            description='Exam Questions',
            button_style='info',
            tooltip='Description',
        )
        self.lesson_plan = widgets.Button(
            description='Lesson Plan',
            button_style='info',
            tooltip='Description',
        )
        self.suggestion_buttons.children = (self.exam_questions, self.lesson_plan )
        self.children = (self.chat_bot, self.suggestion_buttons)

In [ ]:
educator_chatbot = EducatorChatBot(chatbot_model)
educator_chatbot

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()